In [1]:
import numpy as np

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor, SubprocVecEnv
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv

import os
import retro

class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq:
    :param log_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level.
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

        return True

def make_env(env_id, rank, seed=0):
    def _init():
        env = retro.make(game=env_id)
        env = MaxAndSkipEnv(env, 4)
        # env.seed(seed + rank) # seed doesn't want to work... not sure why
        return env

    # set_random_seed(seed)
    return _init

log_dir = 'log/'
os.makedirs(log_dir, exist_ok=True)

env_id = "SuperMarioBros-Nes"
num_cpu = 4


env = VecMonitor(SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)]), log_dir+"monitor")
#model = PPO('CnnPolicy', env, verbose=1, tensorboard_log="./board/", learning_rate=0.00003)

model = PPO.load("./log/best_model.zip", env=env)
print("----START LEARNING----")
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)
model.learn(total_timesteps=1000000, callback=callback, tb_log_name="PPO-00003")
model.save(env_id)
print("----DONE LEARNING----")

2023-06-05 16:03:27.857304: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 16:03:30.313052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 16:03:30.319115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable th

Wrapping the env in a VecTransposeImage.
----START LEARNING----
Logging to ./board/PPO-00003_2
Num timesteps: 4000
Best mean reward: -inf - Last mean reward per episode: 1818.00
Saving new best model to log/best_model
Num timesteps: 8000
Best mean reward: 1818.00 - Last mean reward per episode: 1818.00
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 911      |
|    ep_rew_mean     | 1818.0   |
| time/              |          |
|    fps             | 238      |
|    iterations      | 1        |
|    time_elapsed    | 34       |
|    total_timesteps | 8192     |
---------------------------------
Num timesteps: 12000
Best mean reward: 1818.00 - Last mean reward per episode: 2208.33
Saving new best model to log/best_model
Num timesteps: 16000
Best mean reward: 2208.33 - Last mean reward per episode: 2180.80
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.17e+03     |
|    ep_rew_m

KeyboardInterrupt: 